# Les index

* Auteurs/trices : Adrien VIGUE, Etienne BARON, Camille MONOT

Ce chapitre traite de :
* Index + requêtes textuelles + requêtes géographiques


## À quoi servent-ils ?

Les index prennent en charge l'exécution efficace des requêtes dans MongoDB. Sans index, MongoDB doit effectuer une analyse de collection , c'est-à-dire analyser chaque document d'une collection, pour sélectionner les documents qui correspondent à l'instruction de requête. Si un index approprié existe pour une requête, MongoDB peut utiliser l'index pour limiter le nombre de documents qu'il doit inspecter. Les index de MongoDB sont similaires aux index d'autres systèmes de base de données. MongoDB définit les index au niveau de la collection et prend en charge les index sur n'importe quel champ ou sous-champ des documents dans une collection MongoDB.

En bref, les index sont un moyen de trouver rapidement et efficacement du contenu dans une base de données.

## Quand les utiliser ?

On met en place des index à chaque fois qu'on s'attend à avoir beaucoup de requêtes sur une clé (resp. un ensemble de clés). 
Par exemple, on souhaite récupérer les notes d'un étudiant à l'aide de son numéro étudiant. Il faudra alors mettre un index sur la clé "numéro étudiant" afin des réaliser des requetes efficaces pour retrouver toutes les informations de l'étudiant en fonction de son numéro.

```{admonition} ⚠️ Attention
:class: tip
On ne peut pas toujours utiliser des index. En effet, les index rendent la mise à jour de la base fastidieuse : à chaque fois que vous
rajoutez de nouvelles données, vous devez redéfinir vos index. Il n'est donc pas judicieux d'utiliser des index sur une base où vous vous
attendez à devoir faire des mises à jour régulières.
```

## Syntaxe adaptée

* Création d'un index

```javascript
db.collec.createIndex({"cle":1})
```
On crée ici un index dans la collection 'collec' de la base de données courante 'db'. On donne ensuite le nom du champs sur lequel on va ensuite créer la clé. Le ":1" signifie que l'index va trier les données dans l'ordre croissant.

_Exemple :_

In [1]:
use food

switched to db food

In [2]:
db.NYfood.createIndex({"borough" : 1})

{
	"numIndexesBefore" : 5,
	"numIndexesAfter" : 5,
	"note" : "all indexes already exist",
	"ok" : 1
}

```{admonition} ✍ À noter
Si on avait mis une valeur négative à la place du 1, les données auraient été triées dans l'ordre décroissant. 
Le tri n'a cependant pas d'importance sur l'efficacité de la requête.
```
* Opérateurs bénéficiant de l’index

## Index composés

## Index textuels

## Index géo-spatiaux